<a href="https://colab.research.google.com/github/sabinagio/what-s-the-matter/blob/main/twitter_api_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter API data collection - proof of concept

In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [ ]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAFwFgQEAAAAAOjRNg8PpoZ29uTo2Z%2By0K7DEM3s%3DI1NuBq4xOCsYtDMg8nos58Io1kQKgdW80FSEKaaFcSK69TueFy'

In [ ]:
def auth():
    return os.getenv('TOKEN')

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
def create_url(keyword, max_results=100):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword, 
                    'expansions': 'author_id, in_reply_to_user_id, geo.place_id', 
                    'tweet.fields': 'id, text, author_id, in_reply_to_user_id, geo, conversation_id, created_at, lang, public_metrics, referenced_tweets, reply_settings, source', 
                    'user.fields': 'id, name, username, created_at, description, public_metrics, verified', 
                    'place.fields': 'full_name, id, country, country_code, geo, name, place_type', 
                    'next_token': {}}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token=None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()

In [ ]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "fear lang:en -is:retweet"

In [ ]:
### Small idea space
# perhaps use "I fear", "My fear" 
# remove items that have "fear not"
# How do we find what people fear? 

In [ ]:
url = create_url(keyword)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [ ]:
json_response

{'data': [{'id': '1562715429591601153',
   'source': 'Twitter for iPhone',
   'conversation_id': '1562177791642112000',
   'created_at': '2022-08-25T08:16:06.000Z',
   'referenced_tweets': [{'type': 'replied_to', 'id': '1562177791642112000'}],
   'author_id': '707126315',
   'reply_settings': 'everyone',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': '@vibeforvids FEAR a bad mf lol',
   'lang': 'en',
   'in_reply_to_user_id': '1517401430868459520'},
  {'id': '1562715416094703618',
   'source': 'Twitter for Android',
   'conversation_id': '1562715416094703618',
   'created_at': '2022-08-25T08:16:03.000Z',
   'referenced_tweets': [{'type': 'quoted', 'id': '1562708250067251200'}],
   'author_id': '85720153',
   'reply_settings': 'everyone',
   'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'text': "They are trying to do it by all means necessary. Trigger fea

In [ ]:
# Figure out how to convert the JSON response to Pandas dataframe
json_response.keys()

dict_keys(['data', 'includes', 'meta'])

In [ ]:
json_response["data"][0].keys()

dict_keys(['lang', 'source', 'in_reply_to_user_id', 'conversation_id', 'id', 'public_metrics', 'text', 'created_at', 'author_id', 'reply_settings', 'referenced_tweets'])

In [ ]:
json_response["data"][0]

{'id': '1562715429591601153',
 'source': 'Twitter for iPhone',
 'conversation_id': '1562177791642112000',
 'created_at': '2022-08-25T08:16:06.000Z',
 'referenced_tweets': [{'type': 'replied_to', 'id': '1562177791642112000'}],
 'author_id': '707126315',
 'reply_settings': 'everyone',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'text': '@vibeforvids FEAR a bad mf lol',
 'lang': 'en',
 'in_reply_to_user_id': '1517401430868459520'}

In [ ]:
json_response["includes"]['users'][0].keys()

dict_keys(['created_at', 'username', 'public_metrics', 'description', 'verified', 'id', 'name'])

In [ ]:
json_response["includes"]['users'][0]

{'description': '♍️ ☀️ ♎️ 🌙 ♏️ 🔼  *Loser',
 'username': 'FuckkMasi',
 'created_at': '2013-10-09T23:22:10.000Z',
 'name': 'Beautiful Loser',
 'id': '707126315',
 'public_metrics': {'followers_count': 305,
  'following_count': 695,
  'tweet_count': 10422,
  'listed_count': 4},
 'verified': False}

In [ ]:
json_response["meta"].keys()

dict_keys(['newest_id', 'oldest_id', 'result_count', 'next_token'])

In [ ]:
json_response["meta"]

{'newest_id': '1562715429591601153',
 'oldest_id': '1562715349702754308',
 'result_count': 10,
 'next_token': 'b26v89c19zqg8o3fpz8kqol1bo36239l57j69mqpz4b5p'}

In [ ]:
### CODE FROM AN ARTICLE - TO BE USED FOR INSPIRATION IN GETTING RECURRENT REQUESTS

#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "xbox lang:en"
start_list =    ['2021-01-01T00:00:00.000Z',
                 '2021-02-01T00:00:00.000Z',
                 '2021-03-01T00:00:00.000Z']

end_list =      ['2021-01-31T00:00:00.000Z',
                 '2021-02-28T00:00:00.000Z',
                 '2021-03-31T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)